# How to crunch data from Elasticsearch with python

- Elasticsearch docs: https://www.elastic.co/guide/en/elasticsearch/reference/current/index.html
- Python Elasticsearch docs: https://elasticsearch-py.readthedocs.io/en/master/

In [22]:
import pandas as pd
import requests, json
from elasticsearch import Elasticsearch

pd.set_option('display.max_columns', 150)

## Create the client and connect it to the cluster

In [4]:
host = 'http://35.198.135.86:9200'

es = Elasticsearch(host)

es.info()

{'name': '2a419d875914',
 'cluster_name': 'docker-cluster',
 'cluster_uuid': 'a8qpewqwT2CP-JsWwCMbTg',
 'version': {'number': '7.0.1',
  'build_flavor': 'default',
  'build_type': 'docker',
  'build_hash': 'e4efcb5',
  'build_date': '2019-04-29T12:56:03.145736Z',
  'build_snapshot': False,
  'lucene_version': '8.0.0',
  'minimum_wire_compatibility_version': '6.7.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [5]:
index_0 = 'kibana_sample_data_flights'

index_1 = 'hh_vacancies'

## Match query example ([doc](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-match-query.html))

In [6]:
# All flights from Tokyo Haneda

query = {
    'query': {
        'match': {
            'Origin': 'Tokyo Haneda International Airport'
        }
    }
}

In [7]:
res = es.search(index=index_0, body=query)  # search in the 'Flights' index

sources = [hit['_source'] for hit in res['hits']['hits']]  # fetch only the body of each search hit

In [8]:
df = pd.DataFrame(sources).set_index('FlightNum')

df.head()

,DestCountry,OriginWeather,OriginCityName,AvgTicketPrice,DistanceMiles,FlightDelay,DestWeather,Dest,FlightDelayType,OriginCountry,dayOfWeek,DistanceKilometers,timestamp,DestLocation,DestAirportID,Carrier,Cancelled,FlightTimeMin,Origin,OriginLocation,DestRegion,OriginAirportID,OriginRegion,DestCityName,FlightTimeHour,FlightDelayMin
FlightNum,,,,,,,,,,,,,,,,,,,,,,,,,,
M21BD4I,CN,Cloudy,Tokyo,969.189931,1754.412146,True,Clear,Xi'an Xianyang International Airport,NAS Delay,JP,0,2823.452661,2019-11-04T20:17:26,"{'lat': '34.447102', 'lon': '108.751999'}",XIY,JetBeats,False,261.172633,Tokyo Haneda International Airport,"{'lat': '35.552299', 'lon': '139.779999'}",SE-BD,HND,SE-BD,Xi'an,4.352877,120
KL2UKVD,CH,Cloudy,Tokyo,347.372958,5971.560559,False,Cloudy,Zurich Airport,No Delay,JP,0,9610.295156,2019-11-04T22:34:35,"{'lat': '47.464699', 'lon': '8.54917'}",ZRH,Logstash Airways,False,600.643447,Tokyo Haneda International Airport,"{'lat': '35.552299', 'lon': '139.779999'}",CH-ZH,HND,SE-BD,Zurich,10.010724,0
IR1FR9Z,JP,Sunny,Tokyo,227.668217,0.000000,True,Clear,Tokyo Haneda International Airport,Carrier Delay,JP,0,0.000000,2019-11-04T14:16:01,"{'lat': '35.552299', 'lon': '139.779999'}",HND,JetBeats,False,270.000000,Tokyo Haneda International Airport,"{'lat': '35.552299', 'lon': '139.779999'}",SE-BD,HND,SE-BD,Tokyo,4.500000,270
J0I2OLF,PL,Clear,Tokyo,894.899226,5358.819441,False,Rain,Warsaw Chopin Airport,No Delay,JP,0,8624.183915,2019-11-04T12:43:08,"{'lat': '52.16569901', 'lon': '20.96710014'}",WAW,Logstash Airways,False,479.121329,Tokyo Haneda International Airport,"{'lat': '35.552299', 'lon': '139.779999'}",PL-MZ,HND,SE-BD,Warsaw,7.985355,0
OP4CO7M,AT,Clear,Tokyo,505.499016,5694.907068,True,Thunder & Lightning,Vienna International Airport,Carrier Delay,JP,0,9165.064521,2019-11-04T07:31:56,"{'lat': '48.11029816', 'lon': '16.56970024'}",VIE,Logstash Airways,False,641.004301,Tokyo Haneda International Airport,"{'lat': '35.552299', 'lon': '139.779999'}",AT-9,HND,SE-BD,Vienna,10.683405,30


## Query string query example ([doc](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-query-string-query.html))

In [9]:
# All the vacancies in Minsk or Kiev and with the word "Программист" in the name

query = {
    'query': {
        'query_string': {
            'query': 'address.city:(минск OR киев) AND name:(программист)'
        }
    }
}

In [10]:
res = es.search(index=index_1, body=query)  # search in the 'Vacancies' index

sources = [hit['_source'] for hit in res['hits']['hits']]

In [11]:
df = pd.DataFrame(sources).set_index('id')

df.head()

,premium,name,department,has_test,response_letter_required,area,salary,type,address,response_url,sort_point_distance,employer,published_at,created_at,archived,apply_alternate_url,insider_interview,url,alternate_url,relations,snippet,contacts
id,,,,,,,,,,,,,,,,,,,,,,
34422421,False,Инженер-программист,None,False,False,"{'id': '115', 'name': 'Киев', 'url': 'https://...",None,"{'id': 'open', 'name': 'Открытая'}","{'city': 'Киев', 'street': 'Бориспольская улиц...",None,None,"{'id': '7045', 'name': 'Оптима-фарм, ЛТД', 'ur...",2019-11-05T17:19:45+0300,2019-11-05T17:19:45+0300,False,https://hh.ru/applicant/vacancy_response?vacan...,None,https://api.hh.ru/vacancies/34422421?host=hh.ru,https://hh.ru/vacancy/34422421,[],{'requirement': 'Образование - полное высшее (...,"{'name': 'Нестерова Вита', 'email': 'nesterova..."
34411360,False,Программист РНР,None,False,False,"{'id': '115', 'name': 'Киев', 'url': 'https://...","{'from': None, 'to': 20000, 'currency': 'UAH',...","{'id': 'open', 'name': 'Открытая'}","{'city': 'Киев', 'street': 'улица Ушинского', ...",None,None,"{'id': '3720430', 'name': 'СЕТРА', 'url': 'htt...",2019-11-05T12:10:33+0300,2019-11-05T12:10:33+0300,False,https://hh.ru/applicant/vacancy_response?vacan...,None,https://api.hh.ru/vacancies/34411360?host=hh.ru,https://hh.ru/vacancy/34411360,[],{'requirement': 'Желаемый опыт работы PHP прог...,None
34411360,False,Программист РНР,None,False,False,"{'id': '115', 'name': 'Киев', 'url': 'https://...","{'from': None, 'to': 20000, 'currency': 'UAH',...","{'id': 'open', 'name': 'Открытая'}","{'city': 'Киев', 'street': 'улица Ушинского', ...",None,None,"{'id': '3720430', 'name': 'СЕТРА', 'url': 'htt...",2019-11-05T12:10:33+0300,2019-11-05T12:10:33+0300,False,https://hh.ru/applicant/vacancy_response?vacan...,None,https://api.hh.ru/vacancies/34411360?host=hh.ru,https://hh.ru/vacancy/34411360,[],{'requirement': 'Желаемый опыт работы PHP прог...,None
34422421,False,Инженер-программист,None,False,False,"{'id': '115', 'name': 'Киев', 'url': 'https://...",None,"{'id': 'open', 'name': 'Открытая'}","{'city': 'Киев', 'street': 'Бориспольская улиц...",None,None,"{'id': '7045', 'name': 'Оптима-фарм, ЛТД', 'ur...",2019-11-05T17:19:45+0300,2019-11-05T17:19:45+0300,False,https://hh.ru/applicant/vacancy_response?vacan...,None,https://api.hh.ru/vacancies/34422421?host=hh.ru,https://hh.ru/vacancy/34422421,[],{'requirement': 'Образование - полное высшее (...,"{'name': 'Нестерова Вита', 'email': 'nesterova..."
34526548,False,Программист PHP,None,False,False,"{'id': '115', 'name': 'Киев', 'url': 'https://...",None,"{'id': 'open', 'name': 'Открытая'}","{'city': 'Киев', 'street': 'Сырецко-Печерская ...",None,None,"{'id': '2788383', 'name': 'УКР КРЕДИТ ФИНАНС',...",2019-11-12T11:34:12+0300,2019-11-12T11:34:12+0300,False,https://hh.ru/applicant/vacancy_response?vacan...,None,https://api.hh.ru/vacancies/34526548?host=hh.ru,https://hh.ru/vacancy/34526548,[],{'requirement': 'Специалист уровня middle и вы...,None


## Range query example ([doc](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-range-query.html))

In [12]:
# All flights between 10 days ago from now on up to November, 13

query = {
    'query': {
        'range': {
            'timestamp': {
                # greater/less than (also "gte"/"lte" - greater/less than or equal available)
                # can be defined as ISO8601 timestamp or simply "now-NU", 
                # where N is an integer and U is the unit, e.g. "d" for days
                'gt': 'now-10d',
                'lte': '2019-11-13T06:00:00Z'
            }
        }
    }
}

In [13]:
res = es.search(index=index_0, body=query)  # search in the 'Flights' index

sources = [hit['_source'] for hit in res['hits']['hits']]

In [14]:
df = pd.DataFrame(sources).set_index('FlightNum')

df.head()

,DestCountry,OriginWeather,OriginCityName,AvgTicketPrice,DistanceMiles,FlightDelay,DestWeather,Dest,FlightDelayType,OriginCountry,dayOfWeek,DistanceKilometers,timestamp,DestLocation,DestAirportID,Carrier,Cancelled,FlightTimeMin,Origin,OriginLocation,DestRegion,OriginAirportID,OriginRegion,DestCityName,FlightTimeHour,FlightDelayMin
FlightNum,,,,,,,,,,,,,,,,,,,,,,,,,,
01F7UGE,CN,Thunder & Lightning,Las Vegas,479.965664,6535.199878,False,Sunny,Shanghai Hongqiao International Airport,No Delay,US,3,10517.384712,2019-11-07T21:51:25,"{'lat': '31.19790077', 'lon': '121.3359985'}",SHA,JetBeats,False,876.448726,McCarran International Airport,"{'lat': '36.08010101', 'lon': '-115.1520004'}",SE-BD,LAS,US-NV,Shanghai,14.607479,0
G03IV35,DE,Clear,Dublin,773.129178,677.305794,False,Thunder & Lightning,Frankfurt am Main Airport,No Delay,IE,3,1090.018015,2019-11-07T20:45:42,"{'lat': '50.033333', 'lon': '8.570556'}",FRA,ES-Air,False,68.126126,Dublin Airport,"{'lat': '53.42129898', 'lon': '-6.270070076'}",DE-HE,DUB,IE-D,Frankfurt am Main,1.135435,0
J75BL89,CN,Sunny,Norfolk,366.563175,7607.822454,False,Heavy Fog,Shanghai Pudong International Airport,No Delay,US,3,12243.603420,2019-11-07T12:52:28,"{'lat': '31.14340019', 'lon': '121.8050003'}",PVG,Kibana Airlines,True,720.211966,Norfolk International Airport,"{'lat': '36.89459991', 'lon': '-76.20120239'}",SE-BD,ORF,US-VA,Shanghai,12.003533,0
GBWN5IU,AR,Heavy Fog,Milan,968.406133,6946.527294,False,Cloudy,Ministro Pistarini International Airport,No Delay,IT,3,11179.352022,2019-11-07T19:28:42,"{'lat': '-34.8222', 'lon': '-58.5358'}",EZE,Kibana Airlines,False,657.608942,Malpensa International Airport,"{'lat': '45.6306', 'lon': '8.72811'}",AR-B,MI12,IT-25,Buenos Aires,10.960149,0
IMVN7RN,AR,Sunny,Rome,275.100269,6918.807715,False,Rain,Ministro Pistarini International Airport,No Delay,IT,3,11134.741683,2019-11-07T10:54:35,"{'lat': '-34.8222', 'lon': '-58.5358'}",EZE,Logstash Airways,False,695.921355,Leonardo da Vinci___Fiumicino Airport,"{'lat': '41.8002778', 'lon': '12.2388889'}",AR-B,RM11,IT-62,Buenos Aires,11.598689,0


## Bool query example ([doc](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-bool-query.html))

In [15]:
# Search for all the flight sthat match several criteria: 
# the origin city is Ottawa, the flight was longer than 10 hours and it was flying within the last 15 days from now on

query = {
    'query': {
        'bool': {
            'must': [
                {
                    'match': {
                        'OriginCityName': 'Ottawa'
                    }
                },
                {
                    'query_string': {
                        'query': 'FlightTimeHour:>=10'
                    }
                },
                {
                    'range': {
                        'timestamp': {
                            'gt': 'now-15d',
                            'lte': 'now'
                        }
                    }
                }
            ]
        }
    }
}

In [16]:
res = es.search(index=index_0, body=query)

sources = [hit['_source'] for hit in res['hits']['hits']]

In [17]:
df = pd.DataFrame(sources).set_index('FlightNum')

df.head()

,DestCountry,OriginWeather,OriginCityName,AvgTicketPrice,DistanceMiles,FlightDelay,DestWeather,Dest,FlightDelayType,OriginCountry,dayOfWeek,DistanceKilometers,timestamp,DestLocation,DestAirportID,Carrier,Cancelled,FlightTimeMin,Origin,OriginLocation,DestRegion,OriginAirportID,OriginRegion,DestCityName,FlightTimeHour,FlightDelayMin
FlightNum,,,,,,,,,,,,,,,,,,,,,,,,,,
NRE9J26,AT,Clear,Ottawa,374.899128,4106.857574,False,Rain,Vienna International Airport,No Delay,CA,0,6609.346596,2019-11-04T06:23:10,"{'lat': '48.11029816', 'lon': '16.56970024'}",VIE,Logstash Airways,False,388.785094,Ottawa Macdonald-Cartier International Airport,"{'lat': '45.32249832', 'lon': '-75.66919708'}",AT-9,YOW,CA-ON,Vienna,6.479752,0
7ZTKP49,IT,Rain,Ottawa,893.927742,4036.535456,False,Cloudy,Pisa International Airport,No Delay,CA,0,6496.174118,2019-11-04T13:10:09,"{'lat': '43.683899', 'lon': '10.3927'}",PI05,ES-Air,False,309.341625,Ottawa Macdonald-Cartier International Airport,"{'lat': '45.32249832', 'lon': '-75.66919708'}",IT-52,YOW,CA-ON,Pisa,5.155694,0
5SIPOMV,CN,Heavy Fog,Ottawa,576.047399,7057.750977,False,Cloudy,Shanghai Hongqiao International Airport,No Delay,CA,0,11358.349188,2019-11-04T15:56:09,"{'lat': '31.19790077', 'lon': '121.3359985'}",SHA,Kibana Airlines,False,811.310656,Ottawa Macdonald-Cartier International Airport,"{'lat': '45.32249832', 'lon': '-75.66919708'}",SE-BD,YOW,CA-ON,Shanghai,13.521844,0
1X3WKIL,IT,Sunny,Ottawa,916.421259,3865.611623,False,Heavy Fog,Turin Airport,No Delay,CA,1,6221.098873,2019-11-05T02:40:32,"{'lat': '45.200802', 'lon': '7.64963'}",TO11,Kibana Airlines,False,327.426256,Ottawa Macdonald-Cartier International Airport,"{'lat': '45.32249832', 'lon': '-75.66919708'}",IT-21,YOW,CA-ON,Torino,5.457104,0
UYHN6BP,US,Rain,Ottawa,664.041076,376.858852,True,Clear,Philadelphia International Airport,Carrier Delay,CA,1,606.495532,2019-11-05T14:04:19,"{'lat': '39.87189865', 'lon': '-75.2410965'}",PHL,ES-Air,False,136.920817,Ottawa Macdonald-Cartier International Airport,"{'lat': '45.32249832', 'lon': '-75.66919708'}",US-PA,YOW,CA-ON,Philadelphia,2.282014,105


## Date Histogram aggregation example ([doc](https://www.elastic.co/guide/en/elasticsearch/reference/current/search-aggregations-bucket-datehistogram-aggregation.html))

In [18]:
# How many fligths per day where delayed by more than 2 hours

query = {
    'query': {
        'query_string': {
            'query': 'FlightDelayMin:>120'
        }
    },
    'aggs': {
        'per_day': {
            'date_histogram': {
                'field': 'timestamp',
                'interval': '1d'
            }
        }
    },
    'size': 0  # 'size' is 0 in this case because we don't really need the results themselves - only the time aggregation
}

In [19]:
res = es.search(index=index_0, body=query)

buckets = res['aggregations']['per_day']['buckets']

In [20]:
df = pd.DataFrame(buckets).set_index('key_as_string')

df.head(15)

,key,doc_count
key_as_string,,
2019-11-04T00:00:00.000Z,1572825600000,48
2019-11-05T00:00:00.000Z,1572912000000,47
2019-11-06T00:00:00.000Z,1572998400000,62
2019-11-07T00:00:00.000Z,1573084800000,61
2019-11-08T00:00:00.000Z,1573171200000,57
2019-11-09T00:00:00.000Z,1573257600000,50
2019-11-10T00:00:00.000Z,1573344000000,41
2019-11-11T00:00:00.000Z,1573430400000,63
2019-11-12T00:00:00.000Z,1573516800000,60


## Elastic saved object API ([doc](https://www.elastic.co/guide/en/kibana/master/saved-objects-api.html))

In [23]:
r = requests.get('http://35.198.135.86:5601' + '/api/saved_objects/_find?type=' + 'dashboard', verify=False)

r.json()

{'page': 1,
 'per_page': 20,
 'total': 2,
 'saved_objects': [{'type': 'dashboard',
   'id': '7adfa750-4c81-11e8-b3d7-01146121b73d',
   'attributes': {'title': '[Flights] Global Flight Dashboard',
    'hits': 0,
    'description': 'Analyze mock flight data for ES-Air, Logstash Airways, Kibana Airlines and JetBeats',
    'panelsJSON': '[{"panelIndex":"1","gridData":{"x":0,"y":0,"w":32,"h":7,"i":"1"},"embeddableConfig":{},"version":"6.3.0","panelRefName":"panel_0"},{"panelIndex":"3","gridData":{"x":17,"y":7,"w":23,"h":12,"i":"3"},"embeddableConfig":{"vis":{"colors":{"Average Ticket Price":"#0A50A1","Flight Count":"#82B5D8"},"legendOpen":false}},"version":"6.3.0","panelRefName":"panel_1"},{"panelIndex":"4","gridData":{"x":0,"y":85,"w":48,"h":15,"i":"4"},"embeddableConfig":{},"version":"6.3.0","panelRefName":"panel_2"},{"panelIndex":"5","gridData":{"x":0,"y":7,"w":17,"h":12,"i":"5"},"embeddableConfig":{"vis":{"colors":{"ES-Air":"#447EBC","JetBeats":"#65C5DB","Kibana Airlines":"#BA43A9","Log

## Elastic _cat API ([doc](https://www.elastic.co/guide/en/elasticsearch/reference/current/cat.html))

In [24]:
r = requests.get(host + '/_cat/indices?format=json', verify=False)

r.json()

[{'health': 'green',
  'status': 'open',
  'index': '.kibana_task_manager',
  'uuid': 'jGOO8WSAQ4qp9atSMzyCsQ',
  'pri': '1',
  'rep': '0',
  'docs.count': '2',
  'docs.deleted': '0',
  'store.size': '30.4kb',
  'pri.store.size': '30.4kb'},
 {'health': 'green',
  'status': 'open',
  'index': 'kibana_sample_data_flights',
  'uuid': '4aPdq4ygSRiuNh5JHmJ9oA',
  'pri': '1',
  'rep': '0',
  'docs.count': '13059',
  'docs.deleted': '0',
  'store.size': '6.5mb',
  'pri.store.size': '6.5mb'},
 {'health': 'green',
  'status': 'open',
  'index': '.kibana_1',
  'uuid': 'HxkjfN6oSlSdRgVW25gG4w',
  'pri': '1',
  'rep': '0',
  'docs.count': '103',
  'docs.deleted': '1',
  'store.size': '182.5kb',
  'pri.store.size': '182.5kb'},
 {'health': 'green',
  'status': 'open',
  'index': 'kibana_sample_data_logs',
  'uuid': 'vJOQ-DbeRFmJj6B2fCx5cg',
  'pri': '1',
  'rep': '0',
  'docs.count': '14005',
  'docs.deleted': '0',
  'store.size': '11.5mb',
  'pri.store.size': '11.5mb'},
 {'health': 'yellow',
  'sta

In [25]:
r = requests.get(host + '/_cluster/health', verify=False)

r.json()

{'cluster_name': 'docker-cluster',
 'status': 'yellow',
 'timed_out': False,
 'number_of_nodes': 1,
 'number_of_data_nodes': 1,
 'active_primary_shards': 5,
 'active_shards': 5,
 'relocating_shards': 0,
 'initializing_shards': 0,
 'unassigned_shards': 1,
 'delayed_unassigned_shards': 0,
 'number_of_pending_tasks': 0,
 'number_of_in_flight_fetch': 0,
 'task_max_waiting_in_queue_millis': 0,
 'active_shards_percent_as_number': 83.33333333333334}

In [26]:
r = requests.get(host + '/_nodes/stats', verify=False)

r.json()

{'_nodes': {'total': 1, 'successful': 1, 'failed': 0},
 'cluster_name': 'docker-cluster',
 'nodes': {'r4tce7bDTfqE5160qjWXow': {'timestamp': 1573890380931,
   'name': '2a419d875914',
   'transport_address': '172.18.0.2:9300',
   'host': '172.18.0.2',
   'ip': '172.18.0.2:9300',
   'roles': ['master', 'data', 'ingest'],
   'attributes': {'ml.machine_memory': '15760244736',
    'xpack.installed': 'true',
    'ml.max_open_jobs': '20'},
   'indices': {'docs': {'count': 37570, 'deleted': 1},
    'store': {'size_in_bytes': 38715043},
    'indexing': {'index_total': 37522,
     'index_time_in_millis': 12199,
     'index_current': 0,
     'index_failed': 1,
     'delete_total': 0,
     'delete_time_in_millis': 0,
     'delete_current': 0,
     'noop_update_total': 0,
     'is_throttled': False,
     'throttle_time_in_millis': 0},
    'get': {'total': 4581,
     'time_in_millis': 800,
     'exists_total': 4235,
     'exists_time_in_millis': 725,
     'missing_total': 346,
     'missing_time_in_